1. 클래스 매핑
2. 바운딩 박스 값을 YOLO 형식에 맞게 정규화 (0 ~ 1)

### 클래스 매핑
|middle classification|원본 클래스|변환 클래스|클래스 인덱스|
|---|---|---|---|
|["04"]|화재(불꽃)|발생	화재|0|
|["01", "02", "03"]|검정색/회색/흰색 연기 발생|화재연기|1|
|["05", "06"]|비화재연기/유사 현상|일반연기|2|
|["07" ~ "11"]|구름, 안개, 조명 등|무관객체|3|

In [1]:
import json
import os

# 1. 클래스 매핑 작성
CLASS_MAPPING = {
    "01": 1,  # 화재연기
    "02": 1,  # 화재연기
    "03": 1,  # 화재연기
    "04": 0,  # 화재
    "05": 2,  # 비화재연기
    "06": 2,  # 비화재연기
    "07": 3,  # 무관객체
    "08": 3,  # 무관객체
    "09": 3,  # 무관객체
    "10": 3, # 무관객체
    "11": 3, # 무관객체
}

# 2. COCO 바운딩 박스 -> YOLO 형식으로 정규화
def convert_bbox_to_yolo(box, img_width, img_height):
    """
    COCO 형식의 바운딩 박스를 YOLO 형식으로 변환하고 정규화
    [xmin, ymin, xmax, ymax] -> [x_center, y_center, width, height] (normalized)
    """
    xmin, ymin, xmax, ymax = box

    x_center = (xmin + xmax) / 2.0 # x 중심좌표
    y_center = (ymin + ymax) / 2.0 # y 중심좌표
    width = xmax - xmin   # 가로 길이
    height = ymax - ymin  # 세로 길이
    
    # 정규화
    x_center_norm = x_center / img_width
    y_center_norm = y_center / img_height
    width_norm = width / img_width
    height_norm = height / img_height
    
    return x_center_norm, y_center_norm, width_norm, height_norm

# 3. json파일 -> YOLO 라벨파일
def process_json_file(json_path, output_dir):
    """
    단일 JSON 파일을 처리하여 YOLO 라벨 파일을 생성
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        img_info = data.get("image", {})
        annotations = data.get("annotations", [])
        
        img_filename = img_info.get("filename")
        resolution = img_info.get("resolution")

        # if not img_filename or not resolution:
        #     print(f"경고: 파일 이름 또는 해상도 정보가 누락되었습니다: {json_path}")
        #     return

        img_width, img_height = resolution

        yolo_labels = []
        for ann in annotations:
            middle_class = ann.get("middle classification")
            box = ann.get("box")

            if middle_class in CLASS_MAPPING and box:
                class_id = CLASS_MAPPING[middle_class]
                
                # 바운딩 박스 변환
                yolo_box = convert_bbox_to_yolo(box, img_width, img_height)
                
                # YOLO 형식 문자열 생성
                yolo_labels.append(f"{class_id} {' '.join(map(str, yolo_box))}")

        if yolo_labels:
            # 출력 파일 경로 설정 (원본 파일명에서 확장자만 .txt로 변경)
            base_filename = os.path.splitext(os.path.basename(json_path))[0]
            output_path = os.path.join(output_dir, f"{base_filename}.txt")
            
            # 파일 저장
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write("\n".join(yolo_labels))
            print(f"성공: {json_path} -> {output_path}")

    except Exception as e:
        print(f"오류: 파일을 처리하는 중 오류가 발생했습니다 {json_path} - {e}")

In [5]:
def main():
  # 원본 JSON 파일이 있는 디렉터리
  input_dir_test = "data_raw/test/labels"
  input_dir_train = "data_raw/train/labels"
  input_dir_val = "data_raw/val/labels"
  # 변환된 YOLO 라벨 파일(.txt)을 저장할 디렉터리
  output_dir_test = "data/test/labels"
  output_dir_train = "data/train/labels"
  output_dir_val = "data/val/labels"
  
  # 입력 디렉터리의 모든 JSON 파일에 대해 변환 작업 수행
  print("라벨 변환을 시작합니다...")
  for filename in os.listdir(input_dir_test):
    if filename.endswith(".json"):
      json_path = os.path.join(input_dir_test, filename)
      process_json_file(json_path, output_dir_test)
  print("라벨 변환이 완료되었습니다.")

  # 입력 디렉터리의 모든 JSON 파일에 대해 변환 작업 수행
  print("라벨 변환을 시작합니다...")
  for filename in os.listdir(input_dir_train):
    if filename.endswith(".json"):
      json_path = os.path.join(input_dir_train, filename)
      process_json_file(json_path, output_dir_train)
  print("라벨 변환이 완료되었습니다.")

  # 입력 디렉터리의 모든 JSON 파일에 대해 변환 작업 수행
  print("라벨 변환을 시작합니다...")
  for filename in os.listdir(input_dir_val):
    if filename.endswith(".json"):
      json_path = os.path.join(input_dir_val, filename)
      process_json_file(json_path, output_dir_val)
  print("라벨 변환이 완료되었습니다.")

# 스크립트 실행
main()

라벨 변환을 시작합니다...
성공: data_raw/test/labels/S3-N0401MF00041.json -> data/test/labels/S3-N0401MF00041.txt
성공: data_raw/test/labels/S3-N0401MF00100.json -> data/test/labels/S3-N0401MF00100.txt
성공: data_raw/test/labels/S3-N0401MF00042.json -> data/test/labels/S3-N0401MF00042.txt
성공: data_raw/test/labels/S3-N0401MF00089.json -> data/test/labels/S3-N0401MF00089.txt
성공: data_raw/test/labels/S3-N0401MF00026.json -> data/test/labels/S3-N0401MF00026.txt
성공: data_raw/test/labels/S3-N0401MF00028.json -> data/test/labels/S3-N0401MF00028.txt
성공: data_raw/test/labels/S3-N0401MF00080.json -> data/test/labels/S3-N0401MF00080.txt
성공: data_raw/test/labels/S3-N0401MF00090.json -> data/test/labels/S3-N0401MF00090.txt
성공: data_raw/test/labels/S3-N0401MF00067.json -> data/test/labels/S3-N0401MF00067.txt
성공: data_raw/test/labels/S3-N0401MF00073.json -> data/test/labels/S3-N0401MF00073.txt
성공: data_raw/test/labels/S3-N0401MF00034.json -> data/test/labels/S3-N0401MF00034.txt
성공: data_raw/test/labels/S3-N0401MF000

In [ ]:
import json

json.loads